## Provide a point and size of a bounding box
## determine sample size
## get points and save

In [1]:
import math
import folium
from dotenv import load_dotenv
from os import getenv

import requests
import json

import numpy as np
import pandas as pd

from geopy.distance import ELLIPSOIDS, distance

from datetime import datetime
import sqlite3


In [2]:
test_dir = "d://projects_working_directories//imagery_analysis//20241130_non_pano_images_for_training/"
target_cities = "d://projects_working_directories//imagery_analysis//20241130_non_pano_images_for_training//reference//target_cities.xlsx"

In [3]:
df_target_cities = pd.read_excel(target_cities)
df_target_cities

,country,city,lat,lon,bbox
0,Indonesia,Jakarta,-6.222932,106.833616,50
1,Indonesia,Surabaya,-7.277315,112.729501,50


In [34]:
# User Updated Variables

source_latitude = 34.08147246260309
source_longitude = -118.40090582216759
# this will create a square where each side is 2x the meters
extend_lat_meters = 4_000 
extend_lon_meters = 3_000 

num_samples = 100 # specify the number of random samples you want


In [35]:
source_lat_lon = [source_latitude,source_longitude]

load_dotenv()
API_KEY = getenv("MAPILLARY_CLIENT_TOKEN")

## Establish a location to pull samples from and get target lon/lats

In [36]:
import random

def get_bounding_box(lat, lon, extend_lat_meters, extend_lon_meters):
 # Earth radius in meters
    R = 6378137
    
    # Coordinate offsets in radians
    dLat = extend_lat_meters / R
    dLon = extend_lon_meters / (R * math.cos(math.pi * lat / 180))
    
    # Offset in degrees
    lat_min = lat - dLat * 180 / math.pi
    lat_max = lat + dLat * 180 / math.pi
    lon_min = lon - dLon * 180 / math.pi
    lon_max = lon + dLon * 180 / math.pi


    
    return [lat_min, lat_max, lon_min, lon_max]




bbox = get_bounding_box(source_latitude, source_longitude, extend_lat_meters, extend_lon_meters)



def generate_random_coordinates(bbox, num_samples):

    lat_min = bbox[0]
    lat_max = bbox[1]
    lon_min = bbox[2]
    lon_max = bbox[3]
    
    random_coords = []
    for _ in range(num_samples):
        lat = random.uniform(lat_min, lat_max)
        lon = random.uniform(lon_min, lon_max)
        random_coords.append((lat, lon))
    return random_coords




sample_coordinates = generate_random_coordinates(bbox, num_samples)


In [37]:
## use only if starting from a bbox


In [38]:
def display_map(source_lat_lon, bbox, points=None):

    lat_min = bbox[0]
    lat_max = bbox[1]
    lon_min = bbox[2]
    lon_max = bbox[3]

    
    # Create a map centered around the given coordinates
    m = folium.Map(location=source_lat_lon, zoom_start=11)
    
    # Define the bounding box coordinates
    bounds = [
        [lat_min, lon_min],
        [lat_max, lon_min],
        [lat_max, lon_max],
        [lat_min, lon_max],
        [lat_min, lon_min]  # Close the polygon
    ]
    
    # Add the bounding box to the map
    folium.PolyLine(bounds, color="blue", weight=2.5, opacity=1).add_to(m)

    # Add the center point to the map
    folium.Marker(location=source_lat_lon, popup="Center", icon=folium.Icon(color="red")).add_to(m)
    
    # Add the points to the map
    if points:
        for point in points:
            folium.CircleMarker(location=point, radius=3, color='darkred', fill=True, fill_color='darkred').add_to(m)
        
    return m

map_with_bounding_box = display_map(source_lat_lon, bbox, points=sample_coordinates)
map_with_bounding_box.save("map_with_bounding_box.html")

# Display the map
map_with_bounding_box

## Find actual images on Mapillary that are close to these points

In [39]:

def get_closest_image_id_from_coordinates(access_token, latitude: int, longitude: int, is_pano=None) -> dict:
    #log.debug("Get Image From Coordinates: %s, %s", latitude, longitude, extend_lat_meters=10, extend_lon_meters=10 )
    results = {
        "image_lat": None,
        "image_lon": None,
        "residual": None,
        "image_id": None,
        "camera_type": None,
        "is_pano": None,
        "camera_focal_len": None,
        "camera_k1": None,
        "camera_k2": None,
        "image_path": None,
        "error": None,
    }


    bbox = get_bounding_box(latitude, longitude, 200, 200)
    lat_min = bbox[0]
    lat_max = bbox[1]
    lon_min = bbox[2]
    lon_max = bbox[3]

    mapillary_bbox = f"{lon_min},{lat_min},{lon_max},{lat_max}"


    search_params = {
        "access_token": access_token,
        "fields": "id,thumb_original_url,geometry,camera_type,camera_parameters,is_pano",
        #"fields": "id,thumb_2048_url,geometry",
        #"is_pano": "true",
        "bbox": mapillary_bbox
    }


    #add pano if necessary
    if is_pano == True:
        search_params['is_pano'] = "true"
    elif is_pano == False:
        search_params['is_pano'] = "false"
    else:
        pass


    url = "https://graph.mapillary.com/images"
    try:
        response = requests.get(
            url,
            params=search_params,
        )
        response.raise_for_status()
    except:
        response.raise_for_status()
        print("error")
        return results

    images = response.json()["data"]
    #log.debug("Successfully Retrieved Image Data: %s", images)
    if len(images) == 0:
        #log.debug(
        #    "No Images in Bounding Box: %s", self._bounds(latitude, longitude)
        #)
        return results

    closest = 0
    closest_distance = np.inf

    for i, image in enumerate(images):
    #    filter(lambda img: img["id"] not in self.downloaded_images, images)
    #):
        #print(image)
        image_coordinates = (
            image["geometry"]["coordinates"][1],
            image["geometry"]["coordinates"][0],
        )
        residual = distance(
            (latitude, longitude), image_coordinates, ellipsoid=ELLIPSOIDS["WGS-84"]
        )
        if residual < closest_distance:
            closest = i
            closest_distance = residual

    image = images[closest]
    #log.debug("Closest Image: %s", image["id"])
    results["image_id"] = image["id"]
    results["image_lat"] = image["geometry"]["coordinates"][1]
    results["image_lon"] = image["geometry"]["coordinates"][0]
    results["residual"] = closest_distance.m
    results["image_url"] = image["thumb_original_url"]
    results["camera_type"] = image["camera_type"]
    results["is_pano"] = image["is_pano"]

    camera_params = image.get("camera_parameters")
    if camera_params is not None:
    
        results["camera_focal_len"] = camera_params[0]
        results["camera_k1"] = camera_params[1]
        results["camera_k2"] = camera_params[2]



    return results

In [40]:
def get_image_ids_from_coordinates(access_token, latitude: int, longitude: int, is_pano=None):

    df = pd.DataFrame(columns = ['image_lat','image_lon','residual','image_id','camera_type','is_pano','camera_focal_len','camera_k1','camera_k2','image_path','error','image_url'])
    
    #log.debug("Get Image From Coordinates: %s, %s", latitude, longitude, extend_lat_meters=10, extend_lon_meters=10 )
    results = {
        "image_lat": None,
        "image_lon": None,
        "residual": None,
        "image_id": None,
        "camera_type": None,
        "is_pano": None,
        "camera_focal_len": None,
        "camera_k1": None,
        "camera_k2": None,
        "image_path": None,
        "error": None,
    }


    bbox = get_bounding_box(latitude, longitude, 200, 200)
    lat_min = bbox[0]
    lat_max = bbox[1]
    lon_min = bbox[2]
    lon_max = bbox[3]

    mapillary_bbox = f"{lon_min},{lat_min},{lon_max},{lat_max}"

    search_params = {
                "access_token": access_token,
                "fields": "id,thumb_original_url,geometry,camera_type,camera_parameters,is_pano",
                "limit": 20,
                "bbox": mapillary_bbox
    
    }
    #add pano if necessary
    if is_pano == True:
        search_params['is_pano'] = "true"
    elif is_pano == False:
        search_params['is_pano'] = "false"
    else:
        pass
    

    url = "https://graph.mapillary.com/images"
    try:
        response = requests.get(
            url,
            params=search_params,
        )
        response.raise_for_status()
    except:
        response.raise_for_status()
        print("error")
        return df

    images = response.json()["data"]
    #log.debug("Successfully Retrieved Image Data: %s", images)
    if len(images) == 0:
        #log.debug(
        #    "No Images in Bounding Box: %s", self._bounds(latitude, longitude)
        #)
        return df

    closest = 0
    closest_distance = np.inf

    for i, image in enumerate(images):
    #    filter(lambda img: img["id"] not in self.downloaded_images, images)
    #):
        #print(image)
        image_coordinates = (
            image["geometry"]["coordinates"][1],
            image["geometry"]["coordinates"][0],
        )
        residual = distance(
            (latitude, longitude), image_coordinates, ellipsoid=ELLIPSOIDS["WGS-84"]
        )
        #if residual < closest_distance:
          #  closest = i
         #   closest_distance = residual

        #image = images[closest]
        #log.debug("Closest Image: %s", image["id"])
        results["image_id"] = image["id"]
        results["image_lat"] = image["geometry"]["coordinates"][1]
        results["image_lon"] = image["geometry"]["coordinates"][0]
        results["residual"] = residual.m
        results["image_url"] = image.get("thumb_original_url")
        results["camera_type"] = image.get("camera_type")
        results["is_pano"] = image.get("is_pano")
    
        camera_params = image.get("camera_parameters")
        if camera_params is not None:
        
            results["camera_focal_len"] = camera_params[0]
            results["camera_k1"] = camera_params[1]
            results["camera_k2"] = camera_params[2]
        #else:
        #    results["camera_focal_len"] = 'unavailable'
        #    results["camera_k1"] = 'unavailable'
        #    results["camera_k2"] = 'unavailable'
  
        df.loc[len(df)] = pd.Series(results)
        #tmp_df = pd.DataFrame([results])
        
        #df = pd.concat([df,tmp_df], ignore_index=True)

    if len(df) > 0:
        return df
    return None

In [15]:
df = pd.DataFrame(columns = ['image_lat','image_lon','residual','image_id','camera_type','is_pano','camera_focal_len','camera_k1','camera_k2','image_path','error','image_url'])

for coords in sample_coordinates:
 
    
    tmp_df = get_image_ids_from_coordinates(API_KEY, coords[0], coords[1])
    if isinstance(tmp_df, pd.DataFrame):
        df = pd.concat([df,tmp_df], ignore_index=True)
        #df.iloc[len(df)] = 

In [8]:
bbox

[-6.312763528411952, -6.133100471588048, 106.698070024245, 106.96916197575501]

In [20]:
actual_coords = list(zip(df.image_lat, df.image_lon))
#actual_coords

In [21]:
display_map(source_lat_lon, bbox, points=actual_coords)

In [22]:
print(f"before dedupe {df.shape}")
df = df.drop_duplicates().reset_index(drop=True)
print(f"after dedupe {df.shape}")


before dedupe (180, 12)
after dedupe (180, 12)


In [23]:
timestamp = datetime.now().strftime('%Y-%m-%d')

conn = sqlite3.connect(f'{test_dir}//segmentation_compare_{timestamp}.db')


df.to_sql('sampled_images', con=conn, if_exists='replace', index=False)



# Close the connection
conn.close()

## END

In [74]:
def get_multiple_mapillary_images(api_key, bbox, image_size_indicator='thumb_2048_url'):

    headers = {
        'Authorization': f'OAuth {api_key}'
    }

        #minLon, minLat, maxLon, maxLat
    lat_min = bbox[0]
    lat_max = bbox[1]
    lon_min = bbox[2]
    lon_max = bbox[3]

    mapillary_bbox = f"{lon_min},{lat_min},{lon_max},{lat_max}"


    #for testing
    def _bounds(latitude, longitude) -> str:
        left = longitude - 1000 / 111_111
        bottom = latitude - 1000 / 111_111
        right = longitude + 1000 / 111_111
        top = latitude + 1000 / 111_111
        return f"{left},{bottom},{right},{top}"

    mapillary_bbox = _bounds(3.598224,98.672770)
    print(mapillary_bbox)
    # end for testing

    url = f'https://graph.mapillary.com/images' #?fields=id,bbox={mapillary_bbox},thumb_original_url'
    #url = f'https://graph.mapillary.com/images?fields=id&bbox={minLong},{minLat},{maxLong},{maxLat}'
    #url = 'https://graph.mapillary.com/images?fields=id&bbox=12.967,55.597,13.008,55.607'
    print(url)


    response = requests.get(
        url,
        params={
            "access_token": api_key,
            "fields": "id,thumb_original_url,geometry",
            #"fields": "id,thumb_2048_url,geometry",
            "bbox":mapillary_bbox
        },
    )
    
    #response = requests.get(url, headers=headers)
    print(response)
    response.raise_for_status()

    # Get the image URL and metadata from the response
    data = response.json()
    #print(data)
    #print(response.json())

    return data


data = get_multiple_mapillary_images(API_KEY, bbox)

98.663769991,3.589223990999991,98.681770009,3.607224009000009
https://graph.mapillary.com/images
<Response [200]>
